In [1]:
import requests
import json
import prettytable
import pandas as pd

In [2]:
def get_bls_data(series, start, end):
    headers = {'Content-Type': 'application/json'}
    data = json.dumps({"seriesid": series,"startyear":"%d" % (start), "endyear":"%d" % (end)})
    p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    try:
        df = pd.DataFrame()
        for series in json_data['Results']['series']:
            df_initial = pd.DataFrame(series)
            series_col = df_initial['seriesID'][0]
            for i in range(0, len(df_initial) - 1):
                df_row = pd.DataFrame(df_initial['data'][i])
                df_row['seriesID'] = series_col
                if 'code' not in str(df_row['footnotes']): 
                    df_row['footnotes'] = ''
                else:
                    df_row['footnotes'] = str(df_row['footnotes']).split("'code': '",1)[1][:1]
                df = df.append(df_row, ignore_index=True)
        return df
    except:
        json_data['status'] == 'REQUEST_NOT_PROCESSED'
        print('BLS API has given the following Response:', json_data['status'])
        print('Reason:', json_data['message'])



In [4]:

start = 2019
end = 2020
series = [# (Unadj) Civilian Labor Force Level - Less than a High School Diploma, 25 yrs. & over
          'LNU01027659', 
          # (Unadj) Unemployment Level - Less than a High School Diploma, 25 yrs. & over
          'LNU03027659', 
          # (Unadj) Unemployment Rate - Less than a High School Diploma, 25 yrs. & over
          'LNU04027659', 
          # (Unadj) Civilian Labor Force Level - High School Graduates, No College, 25 yrs. & over
          'LNU01027660', 
          # (Unadj) Unemployment Level - High School Graduates, No College, 25 yrs. & over
          'LNU03027660', 
          # (Unadj) Unemployment Rate - High School Graduates, No College, 25 yrs. & over
          'LNU04027660', 
          # (Unadj) Civilian Labor Force Level - Some College or Associate Degree, 25 yrs. & over
          'LNU01027689', 
          # (Unadj) Unemployment Level - Some College or Associate Degree, 25 yrs. & over
          'LNU03027689', 
          # (Unadj) Unemployment Rate - Some College or Associate Degree, 25 yrs. & over
          'LNU04027689', 
          # (Unadj) Civilian Labor Force Level - Bachelor's degree and higher, 25 yrs. & over
          'LNU01027662', 
          # (Unadj) Unemployment Level - Bachelor's degree and higher, 25 yrs. & over
          'LNU03027662', 
          # (Unadj) Unemployment Rate - Bachelor's degree and higher, 25 yrs. & over
          'LNU04027662']

df = get_bls_data(series=series, start=start, end=end)
writer = pd.ExcelWriter('bls2-education.xlsx', engine='xlsxwriter', options={'strings_to_numbers': True})
df.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()